In [2]:
import Pkg; Pkg.add("PowerModels")
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Statistics")
Pkg.add("JuMP")
Pkg.add("Gurobi")
Pkg.add("LinearAlgebra")
Pkg.add("FileIO")
Pkg.add("Distributions")
Pkg.add("Tables")
Pkg.add("SparseArrays")
Pkg.instantiate()
Pkg.add("JLD2")

   Resolving package versions...
  No Changes to `/project/dilkina_438/weiminhu/miplib/generator_ryan/lp_generator/Project.toml`
  No Changes to `/project/dilkina_438/weiminhu/miplib/generator_ryan/lp_generator/Manifest.toml`
   Resolving package versions...
  No Changes to `/project/dilkina_438/weiminhu/miplib/generator_ryan/lp_generator/Project.toml`
  No Changes to `/project/dilkina_438/weiminhu/miplib/generator_ryan/lp_generator/Manifest.toml`
   Resolving package versions...
  No Changes to `/project/dilkina_438/weiminhu/miplib/generator_ryan/lp_generator/Project.toml`
  No Changes to `/project/dilkina_438/weiminhu/miplib/generator_ryan/lp_generator/Manifest.toml`
   Resolving package versions...
  No Changes to `/project/dilkina_438/weiminhu/miplib/generator_ryan/lp_generator/Project.toml`
  No Changes to `/project/dilkina_438/weiminhu/miplib/generator_ryan/lp_generator/Manifest.toml`
   Resolving package versions...
  No Changes to `/project/dilkina_438/weiminhu/miplib/generator

In [3]:
# Select the project
cd("/project/dilkina_438/weiminhu/miplib/generator_ryan/lp_generator")

# Set environment
using Pkg
Pkg.activate(".")
Pkg.resolve()
Pkg.update()

#load packages
using PowerModels
using CSV
using DataFrames
using Statistics
using JuMP
using Gurobi
using LinearAlgebra
using Printf
using FileIO
using Dates
using Distributions
using Tables
using SparseArrays
using JLD2

#import functions
include("src_sharing/add_DCOPF_constraints.jl")
include("src_sharing/add_objective.jl")
include("src_sharing/add_variables.jl")
include("src_sharing/calc_cost.jl")
include("src_sharing/fire_risk_lines.jl")
include("src_sharing/save_results.jl")
include("src_sharing/preprocessing.jl")
include("src_sharing/generate_loads.jl")
include("src_sharing/PSPS_undergrounding_optimization.jl")
include("src_sharing/get_line_lengths.jl")

#ENV["GRB_LICENSE_FILE"]="/usr/local/pace-apps/manual/packages/gurobi/9.5.0/license/custom_grbd_2022-10-07.lic"

#turn off notifications in PowerModels
PowerModels.silence()

  Activating project at `/project/dilkina_438/weiminhu/miplib/generator_ryan/lp_generator`
  No Changes to `/project/dilkina_438/weiminhu/miplib/generator_ryan/lp_generator/Project.toml`
  No Changes to `/project/dilkina_438/weiminhu/miplib/generator_ryan/lp_generator/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `/project/dilkina_438/weiminhu/miplib/generator_ryan/lp_generator/Project.toml`
  No Changes to `/project/dilkina_438/weiminhu/miplib/generator_ryan/lp_generator/Manifest.toml`


[info | PowerModels]: Suppressing information and warning messages for the rest of this session.  Use the Memento package for more fine-grained control of logging.


In [4]:
#################################################################
#Import Network
#################################################################
# network_dict = Dict(
#     "Texas7k" => Dict(
#         :network_data => PowerModels.parse_file("data/networks/Texas7k_20210804.m"),
#         :busdata => DataFrame(CSV.File("data/bus_lat_lons/Texas7k_lat_long.csv"))
#         # :ref => PowerModels.build_ref(network_data)[:it][:pm][:nw][0],
#         )
#     )

network_dict = Dict(
    "Texas7k" => Dict(
        :network_data => PowerModels.parse_file("data/networks/Texas7k_20210804.m"),
        :busdata => DataFrame(CSV.File("data/bus_lat_lons/Texas7k_lat_long.csv"))
        # :ref => PowerModels.build_ref(network_data)[:it][:pm][:nw][0],
        ),
    "WECC240" => Dict(
        :network_data => PowerModels.parse_file("data/networks/pglib_opf_case240_pserc.m"),
        :busdata => DataFrame(CSV.File("data/bus_lat_lons/wecc_lat_lon_good.csv"))
        # :ref => PowerModels.build_ref(network_data)[:it][:pm][:nw][0],
        )

    )
#################################################################
#Set Problem Parameters
#################################################################
#network_names = ["Texas7k"]

network_names = ["WECC240"]

#establish budget for undergrounding--make sure this is float
Bs = 300.0:5.0:500.0
#Bs = 0.0:100.0:300.0
#Bs = 4.76:0.01:5.40
# risk threshold; this is currently constant, but it could be variable for each line
# These are individual line threshold, anything with risk under 1e-4 will remain on. 
#Anything above 1 must be off or undergrounded
taus=[[1e-4, 1.0]] 

#This is total network wide threshold. Recommended range is 300-800
thresholds = [600.0]

#year of simulations
simulation_year = 2021

#start date of simulations
month_start = 6
day_start  = 5

#end date of simulations
month_end = 6
day_end =20 #12,11

#calculate number of days in simulation
start_date = Date(simulation_year, month_start, day_start)
end_date = Date(simulation_year, month_end, day_end)
num_days = Dates.value(end_date - start_date) + 1   

#number of days in forecast horizon
forecast_horizon = 1
D = num_days

#number of switchable lines
num_switchable_lines = Inf

#set the optimization method
obj = "wildfire"

#number of time periods per day
T = 1

use_warm_start = false # NOT USING WARMSTART HEURISTIC

# undergrounding cost per mile
cost_per_mile = 7 #million

#allowed runtime in seconds of optimization algorithm (make sure this is float)
MIP_timelim = 60000.0

60000.0

In [ ]:
###############################################################################################################################################################
# SCRIPT BEGINS HERE; ALL PARAMETER CHANGES TAKE PLACE ABOVE THIS LINE
###############################################################################################################################################################

#################################################################
#Run Pre-processing
#################################################################
load_shed_dict = Dict()
load_demanded_dict = Dict()

opt_parameters = Dict()
results = Dict()

#Number of scenarios to generation
#N = 5000
#N = 10000
N = 50
# IF SET TO true, will only generate lp files. If set to false, will solve optimzation problem
lp_only = true 

for network_name in network_names
    network_data = network_dict[network_name][:network_data]
    busdata = network_dict[network_name][:busdata]
    ref = PowerModels.build_ref(network_data)[:it][:pm][:nw][0]

    branch_names = sort([branch for branch in keys(ref[:branch])])
    bus_names = sort([bus for bus in keys(ref[:bus])])
    
    results = Dict()

    # get costs for undergrounding
    undergrounding_costs = Dict()
    line_lengths = calculate_line_lengths(ref, busdata)
    for i in keys(line_lengths)
        cost = line_lengths[i] * cost_per_mile
        push!(undergrounding_costs, i => cost)
    end

    
    for tau in taus

        # make tau dictionary, threshold per line
        tau_dict_up = Dict()
        tau_dict_mid = Dict()
        for branch in branch_names
            push!(tau_dict_up, branch => tau[2])
            push!(tau_dict_mid, branch => tau[1])
        end

        for B in Bs
            for threshold in thresholds #network thresholds
                for n in 1:N
                    name_of_model = network_name

                    println("Running for network $name_of_model budget $B million, network threshold $threshold, and days $D.")

                    # run pre-processing code
                    opt_parameters = preprocessing(name_of_model, network_data, simulation_year, month_start, day_start, month_end, day_end, T, tau_dict_up, tau_dict_mid, threshold)

                    ref = opt_parameters[:ref]
                    branch_names = sort([branch for branch in ref[:arcs_from]])
                    println("pre-processing done")

                    #################################################################
                    #Run Warm Start
                    #################################################################
                    warm_start_dict = Dict()
                    
                    #################################################################
                    #Run Optimization
                    #################################################################

                    #run optimization
                    save_trace =""
                    mkpath("/project/dilkina_438/weiminhu/miplib/generator_ryan/instances/small/$(D)_day/budget_$(B)")
                    #mkpath("/project/dilkina_438/weiminhu/miplib/generator_ryan/instances/$(D)_day/budget_$(B)/test")
                    save_lp = "/project/dilkina_438/weiminhu/miplib/generator_ryan/instances/small/$(D)_day/budget_$(B)/$(name_of_model)_days_$(D)_threshold_$(threshold)_$(month_start)-$(day_start)_$(month_end)-$(day_end)_$(n).lp"
                    #save_lp = "/project/dilkina_438/weiminhu/miplib/generator_ryan/instances/$(D)_day/budget_$(B)/test/$(name_of_model)_days_$(D)_threshold_$(threshold)_$(month_start)-$(day_start)_$(month_end)-$(day_end)_$(n).lp"
                    results = wildfire_undergrounding_opt(opt_parameters, opt_parameters[:actual_load], D, undergrounding_costs, B, warm_start_dict, MIP_timelim, lp_only, log_str = save_trace, lp_str = save_lp)  
                    
                end
            end
        end
    end
end


In [5]:
Pkg.status()

Status `/project/dilkina_438/weiminhu/miplib/generator_ryan/lp_generator/Project.toml`
  [336ed68f] CSV v0.10.13
  [a93c6f00] DataFrames v1.6.1
  [31c24e10] Distributions v0.25.107
  [5789e2e9] FileIO v1.16.3
  [2e9cd046] Gurobi v1.2.3
  [033835bb] JLD2 v0.4.46
  [4076af6c] JuMP v1.21.0
  [c36e90e8] PowerModels v0.21.1
  [bd369af6] Tables v1.11.1
  [37e2e46d] LinearAlgebra
  [2f01184e] SparseArrays
  [10745b16] Statistics v1.9.0


In [32]:
using Distributions

value = rand(Uniform(0.95,1.05),1,1)[1]

1.011534221209878